## Part a - Download the geometries from the source 

In [8]:
import wget

print('Beginning file download with wget module')

url = 'http://raw.githubusercontent.com/datasets/geo-countries/master/data/countries.geojson'
wget.download(url, 'countries.geojson')


Beginning file download with wget module


'countries (1).geojson'

In [4]:
# import geojson
# with open('countries.geojson') as f:
#     gj = geojson.load(f)
# features = gj['features'][0]
# features


{"geometry": {"coordinates": [[[[-69.996938, 12.577582], [-69.936391, 12.531724], [-69.924672, 12.519232], [-69.915761, 12.497016], [-69.880198, 12.453559], [-69.87682, 12.427395], [-69.888092, 12.41767], [-69.908803, 12.417792], [-69.930531, 12.425971], [-69.945139, 12.440375], [-69.924672, 12.440375], [-69.924672, 12.447211], [-69.958567, 12.463202], [-70.027659, 12.522935], [-70.048085, 12.531155], [-70.058095, 12.537177], [-70.062408, 12.54682], [-70.060374, 12.556952], [-70.051096, 12.574042], [-70.048736, 12.583726], [-70.052642, 12.600002], [-70.059641, 12.614244], [-70.061106, 12.625393], [-70.048736, 12.632148], [-70.007151, 12.585517], [-69.996938, 12.577582]]]], "type": "MultiPolygon"}, "properties": {"ADMIN": "Aruba", "ISO_A2": "AW", "ISO_A3": "ABW"}, "type": "Feature"}

## Using ogr2ogr to import geojson file into POSTGIS
Terminal Command:

ogr2ogr -f PostgreSQL PG:"dbname=positiumDB user=enlik" countries.geojson -nln countries -nlt MULTIPOLYGON

and then I export table 'countries' into countries.csv file using 'Postico' Mac OS application

## Read CSV

In [ ]:
import pandas as pd
import psycopg2
import psycopg2.extras

In [5]:
countries = pd.read_csv("countries.csv")

In [6]:
countries.head()

,ogc_fid,admin,iso_a3,iso_a2,wkb_geometry
0,1,Aruba,ABW,AW,0106000020E6100000010000000103000000010000001A...
1,2,Afghanistan,AFG,AF,0106000020E610000001000000010300000001000000FD...
2,3,Angola,AGO,AO,0106000020E6100000030000000103000000010000000E...
3,4,Anguilla,AIA,AI,0106000020E61000000200000001030000000100000018...
4,5,Albania,ALB,AL,0106000020E6100000010000000103000000010000002D...


In [7]:
countries = countries.filter(['ogc_fid', 'wkb_geometry'])
countries

,ogc_fid,wkb_geometry
0,1,0106000020E6100000010000000103000000010000001A...
1,2,0106000020E610000001000000010300000001000000FD...
2,3,0106000020E6100000030000000103000000010000000E...
3,4,0106000020E61000000200000001030000000100000018...
4,5,0106000020E6100000010000000103000000010000002D...
...,...,...
250,251,0106000020E61000000200000001030000000100000037...
251,252,0106000020E61000000900000001030000000100000009...
252,253,0106000020E6100000030000000103000000010000001A...
253,254,0106000020E610000001000000010300000001000000E4...


In [116]:
countries = countries.rename(columns = {"ogc_fid": "geom_id", "wkb_geometry": "geom"})

In [117]:
countries

,geom_id,geom
0,1,0106000020E6100000010000000103000000010000001A...
1,2,0106000020E610000001000000010300000001000000FD...
2,3,0106000020E6100000030000000103000000010000000E...
3,4,0106000020E61000000200000001030000000100000018...
4,5,0106000020E6100000010000000103000000010000002D...
...,...,...
250,251,0106000020E61000000200000001030000000100000037...
251,252,0106000020E61000000900000001030000000100000009...
252,253,0106000020E6100000030000000103000000010000001A...
253,254,0106000020E610000001000000010300000001000000E4...


In [118]:
df_columns = list(countries)
df_columns

columns = ",".join(df_columns)
columns

'geom_id,geom'

In [119]:
values = "VALUES({})".format(",".join(["%s" for _ in df_columns]))
values

'VALUES(%s,%s)'

In [120]:
table = 'test_gis.geom'
table

'test_gis.geom'

In [121]:
conn = psycopg2.connect("host=localhost dbname=positiumDB user=enlik")
cur = conn.cursor()

insert_stmt = "INSERT INTO {} ({}) {}".format(table,columns,values)

psycopg2.extras.execute_batch(cur, insert_stmt, countries.values)
conn.commit()

In [21]:
a2_list = pd.read_csv("iso_a2_list_new.csv")
a2_list = a2_list.filter(['iso_a2', 'country_name_eng', 'geom_id'])
a2_list.head()

,iso_a2,country_name_eng,geom_id
0,ae,United Arab Emirates,NaN
1,af,Afghanistan,NaN
2,ag,Antigua and Barbuda,NaN
3,ai,Anguilla,NaN
4,al,Albania,NaN


In [9]:
countries_new = pd.read_csv("countries.csv")
countries_new = countries_new.filter(['ogc_fid', 'iso_a2'])
countries_new['iso_a2'] = countries_new['iso_a2'].str.lower()
countries_new = countries_new.rename(columns = {"ogc_fid": "geom_id"})
countries_new.head(20)

,geom_id,iso_a2
0,1,aw
1,2,af
2,3,ao
3,4,ai
4,5,al
5,6,ax
6,7,ad
7,8,ae
8,9,ar
9,10,am


In [22]:
# result = pd.merge(a2_list, countries_new, how = 'left', on = 'iso_a2')
result = pd.merge(a2_list, countries_new, on = 'iso_a2')
result = result.drop(['geom_id_x'], axis = 1)
result = result.rename(columns = {'geom_id_y': "geom_id"})
result.head(20)

,iso_a2,country_name_eng,geom_id
0,ae,United Arab Emirates,8
1,af,Afghanistan,2
2,ag,Antigua and Barbuda,15
3,ai,Anguilla,4
4,al,Albania,5
5,am,Armenia,10
6,ao,Angola,3
7,aq,Antarctica,12
8,aq,Antarctica,13
9,ar,Argentina,9


In [23]:
df_columns = list(result)
df_columns

columns = ",".join(df_columns)
columns

'iso_a2,country_name_eng,geom_id'

In [24]:
values = "VALUES({})".format(",".join(["%s" for _ in df_columns]))
values

'VALUES(%s,%s,%s)'

In [25]:
table = 'test_gis.iso_a2_list_new'
table

'test_gis.iso_a2_list_new'

In [26]:
conn = psycopg2.connect("host=localhost dbname=positiumDB user=enlik")
cur = conn.cursor()

cur.execute("""
CREATE TABLE test_gis.iso_a2_list_new (id serial primary key, iso_a2 varchar(2), country_name_eng varchar(50), geom_id int)
""")
conn.commit()

DuplicateTable: relation "iso_a2_list_new" already exists


In [30]:
insert_stmt = "INSERT INTO {} ({}) {}".format(table,columns,values)

psycopg2.extras.execute_batch(cur, insert_stmt, result.values)
conn.commit()